In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
from tensorflow.python.keras.backend import set_session
import tensorflow as tf
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2)  
from tensorflow.keras.layers import Dense, MaxPooling1D, Flatten
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.compat.v1.keras.backend as K
from tensorflow.keras.models import load_model
from tcn import TCN, tcn_full_summary
from tcn import compiled_tcn
from tensorflow.keras.utils import to_categorical
import csv
import random
import itertools
import time
import pickle
from data_utils import *

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6



## Import Dataset

In [2]:
sampling_rate = 100
window_size = 550
stride = 50
f = '/home/nesl/209as_sec/human_act/Data/Activity_Dataset/'

X_tr, Y_tr, X_test, Y_test = import_auritus_activity_dataset(dataset_folder = f, 
                                use_timestamp=False, 
                                shuffle=True, 
                                window_size = window_size, stride = stride, 
                                return_test_set = True, test_set_size = 300,channels=0)

print(X_tr.shape)
print(Y_tr.shape)
print(X_test.shape)
print(Y_test.shape)

100%|█████████████████████████████████████████| 399/399 [00:13<00:00, 29.41it/s]


(14671, 550, 6)
(14671, 9)
(300, 550, 6)
(300, 9)


# Training

In [3]:
model_name = 'Auritus_TCN.h5'
model_epochs = 600 #epochs to train each model for

In [4]:
input_dim=X_tr.shape[2]

model = compiled_tcn(return_sequences=False,
                     num_feat=input_dim,
                     num_classes=Y_tr.shape[1],
                     nb_filters=21,
                     kernel_size=2,
                     dilations=[2,8,16,64,128,256],
                     nb_stacks=1,
                     max_len=window_size,
                     use_weight_norm=False,
                     use_skip_connections=True)

opt = tf.keras.optimizers.Adam()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=opt,metrics=['accuracy'])
model.summary()

x.shape= (None, 21)
model.x = (None, 550, 6)
model.y = (None, 9)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 550, 6)]          0         
_________________________________________________________________
tcn (TCN)                    (None, 21)                10353     
_________________________________________________________________
dense (Dense)                (None, 9)                 198       
_________________________________________________________________
activation (Activation)      (None, 9)                 0         
Total params: 10,551
Trainable params: 10,551
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpoint = ModelCheckpoint(model_name, monitor='val_accuracy', verbose=1, save_best_only=True)
model.fit(x=X_tr, y=Y_tr,validation_split=0.1,
          epochs=model_epochs,callbacks=[checkpoint],shuffle=True,verbose=1)

Epoch 1/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1289 - accuracy: 0.9530 - val_loss: 0.2510 - val_accuracy: 0.9196

Epoch 00001: val_accuracy improved from -inf to 0.91962, saving model to Auritus_TCN.h5
Epoch 2/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1310 - accuracy: 0.9536 - val_loss: 0.2464 - val_accuracy: 0.9108

Epoch 00002: val_accuracy did not improve from 0.91962
Epoch 3/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1299 - accuracy: 0.9525 - val_loss: 0.2476 - val_accuracy: 0.9149

Epoch 00003: val_accuracy did not improve from 0.91962
Epoch 4/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1272 - accuracy: 0.9533 - val_loss: 0.2304 - val_accuracy: 0.9257

Epoch 00004: val_accuracy improved from 0.91962 to 0.92575, saving model to Auritus_TCN.h5
Epoch 5/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1293 - accuracy: 0.9530 - val_loss: 0.2424 - val_accuracy:


Epoch 00041: val_accuracy did not improve from 0.92575
Epoch 42/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1209 - accuracy: 0.9569 - val_loss: 0.2541 - val_accuracy: 0.9176

Epoch 00042: val_accuracy did not improve from 0.92575
Epoch 43/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1261 - accuracy: 0.9548 - val_loss: 0.2519 - val_accuracy: 0.9169

Epoch 00043: val_accuracy did not improve from 0.92575
Epoch 44/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1155 - accuracy: 0.9583 - val_loss: 0.2661 - val_accuracy: 0.9183

Epoch 00044: val_accuracy did not improve from 0.92575
Epoch 45/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1189 - accuracy: 0.9568 - val_loss: 0.2839 - val_accuracy: 0.9135

Epoch 00045: val_accuracy did not improve from 0.92575
Epoch 46/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1212 - accuracy: 0.9554 - val_loss: 0.2640 - val_accuracy: 0.9189


413/413 [==============================] - 4s 9ms/step - loss: 0.1150 - accuracy: 0.9567 - val_loss: 0.2729 - val_accuracy: 0.9128

Epoch 00082: val_accuracy did not improve from 0.93529
Epoch 83/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1158 - accuracy: 0.9576 - val_loss: 0.2623 - val_accuracy: 0.9196

Epoch 00083: val_accuracy did not improve from 0.93529
Epoch 84/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1141 - accuracy: 0.9597 - val_loss: 0.2531 - val_accuracy: 0.9162

Epoch 00084: val_accuracy did not improve from 0.93529
Epoch 85/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1123 - accuracy: 0.9608 - val_loss: 0.2519 - val_accuracy: 0.9183

Epoch 00085: val_accuracy did not improve from 0.93529
Epoch 86/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1183 - accuracy: 0.9575 - val_loss: 0.2604 - val_accuracy: 0.9278

Epoch 00086: val_accuracy did not improve from 0.93529
Epoch 87/300


413/413 [==============================] - 4s 9ms/step - loss: 0.1116 - accuracy: 0.9608 - val_loss: 0.2701 - val_accuracy: 0.9189

Epoch 00123: val_accuracy did not improve from 0.93529
Epoch 124/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1082 - accuracy: 0.9630 - val_loss: 0.2728 - val_accuracy: 0.9237

Epoch 00124: val_accuracy did not improve from 0.93529
Epoch 125/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1111 - accuracy: 0.9604 - val_loss: 0.2671 - val_accuracy: 0.9223

Epoch 00125: val_accuracy did not improve from 0.93529
Epoch 126/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1040 - accuracy: 0.9617 - val_loss: 0.2518 - val_accuracy: 0.9257

Epoch 00126: val_accuracy did not improve from 0.93529
Epoch 127/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1117 - accuracy: 0.9605 - val_loss: 0.2598 - val_accuracy: 0.9257

Epoch 00127: val_accuracy did not improve from 0.93529
Epoch 128


Epoch 00163: val_accuracy did not improve from 0.93869
Epoch 164/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1093 - accuracy: 0.9603 - val_loss: 0.2747 - val_accuracy: 0.9251

Epoch 00164: val_accuracy did not improve from 0.93869
Epoch 165/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1089 - accuracy: 0.9614 - val_loss: 0.2446 - val_accuracy: 0.9271

Epoch 00165: val_accuracy did not improve from 0.93869
Epoch 166/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1076 - accuracy: 0.9607 - val_loss: 0.2533 - val_accuracy: 0.9264

Epoch 00166: val_accuracy did not improve from 0.93869
Epoch 167/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1087 - accuracy: 0.9624 - val_loss: 0.2795 - val_accuracy: 0.9183

Epoch 00167: val_accuracy did not improve from 0.93869
Epoch 168/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1196 - accuracy: 0.9603 - val_loss: 0.2460 - val_accuracy: 0.

413/413 [==============================] - 3s 8ms/step - loss: 0.1157 - accuracy: 0.9604 - val_loss: 0.2600 - val_accuracy: 0.9210

Epoch 00204: val_accuracy did not improve from 0.93869
Epoch 205/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1070 - accuracy: 0.9620 - val_loss: 0.2517 - val_accuracy: 0.9203

Epoch 00205: val_accuracy did not improve from 0.93869
Epoch 206/300
413/413 [==============================] - 3s 8ms/step - loss: 0.1064 - accuracy: 0.9624 - val_loss: 0.2370 - val_accuracy: 0.9305

Epoch 00206: val_accuracy did not improve from 0.93869
Epoch 207/300
413/413 [==============================] - 3s 8ms/step - loss: 0.1017 - accuracy: 0.9642 - val_loss: 0.2423 - val_accuracy: 0.9217

Epoch 00207: val_accuracy did not improve from 0.93869
Epoch 208/300
413/413 [==============================] - 3s 8ms/step - loss: 0.1173 - accuracy: 0.9608 - val_loss: 0.2584 - val_accuracy: 0.9210

Epoch 00208: val_accuracy did not improve from 0.93869
Epoch 209

413/413 [==============================] - 4s 8ms/step - loss: 0.1081 - accuracy: 0.9615 - val_loss: 0.2661 - val_accuracy: 0.9257

Epoch 00245: val_accuracy did not improve from 0.93869
Epoch 246/300
413/413 [==============================] - 3s 8ms/step - loss: 0.1078 - accuracy: 0.9611 - val_loss: 0.2649 - val_accuracy: 0.9196

Epoch 00246: val_accuracy did not improve from 0.93869
Epoch 247/300
413/413 [==============================] - 3s 8ms/step - loss: 0.0985 - accuracy: 0.9656 - val_loss: 0.2568 - val_accuracy: 0.9223

Epoch 00247: val_accuracy did not improve from 0.93869
Epoch 248/300
413/413 [==============================] - 4s 9ms/step - loss: 0.1028 - accuracy: 0.9649 - val_loss: 0.2518 - val_accuracy: 0.9230

Epoch 00248: val_accuracy did not improve from 0.93869
Epoch 249/300
413/413 [==============================] - 3s 8ms/step - loss: 0.1029 - accuracy: 0.9643 - val_loss: 0.2584 - val_accuracy: 0.9149

Epoch 00249: val_accuracy did not improve from 0.93869
Epoch 250

413/413 [==============================] - 3s 8ms/step - loss: 0.1025 - accuracy: 0.9666 - val_loss: 0.2547 - val_accuracy: 0.9244

Epoch 00286: val_accuracy did not improve from 0.93869
Epoch 287/300
413/413 [==============================] - 3s 8ms/step - loss: 0.0932 - accuracy: 0.9663 - val_loss: 0.2442 - val_accuracy: 0.9305

Epoch 00287: val_accuracy did not improve from 0.93869
Epoch 288/300
413/413 [==============================] - 3s 8ms/step - loss: 0.1098 - accuracy: 0.9633 - val_loss: 0.2579 - val_accuracy: 0.9203

Epoch 00288: val_accuracy did not improve from 0.93869
Epoch 289/300
413/413 [==============================] - 4s 9ms/step - loss: 0.0953 - accuracy: 0.9670 - val_loss: 0.2525 - val_accuracy: 0.9292

Epoch 00289: val_accuracy did not improve from 0.93869
Epoch 290/300
413/413 [==============================] - 3s 8ms/step - loss: 0.0936 - accuracy: 0.9667 - val_loss: 0.2611 - val_accuracy: 0.9237

Epoch 00290: val_accuracy did not improve from 0.93869
Epoch 291

## Evaluate Model

In [7]:
model = load_model(model_name,custom_objects={'TCN': TCN})
test_accu = model.evaluate(x=X_test,y=Y_test)[1]
print('Test Accuracy:', test_accu)

10/10 [==============================] - 0s 7ms/step - loss: 0.2979 - accuracy: 0.9300
Test Accuracy: 0.9300000071525574
